# **과제2 : 챗봇 모델 완성하기**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [30]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [ ]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [62]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

In [7]:
data = pd.read_csv('aivleschool_qa.csv', encoding='utf-8')
data.head()

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\nKT 에이블스쿨은 정규 4년제...
1,모집/선발,35세 이상은 지원할 수 없나요?\n본 교육 과정은 34세 이하를 대상으로 하는 교...
2,모집/선발,미취업자의 기준이 뭔가요?\n미취업자의 기준은 아래와 같습니다.\n1) 기간의 정함...
3,모집/선발,"직장인도 지원할 수 있나요?\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 1...


* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./database



In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
database = Chroma(persist_directory="./db_chatlog", embedding_function = embeddings )

* 데이터 입력
    * 이미 입력된 데이터는 제외하고 나머지 입력하기

In [8]:
# 각 행의 데이터를 Document 객체로 변환 
documents = [Document(page_content=text) for text in data['QA'].tolist()]

# 데이터프레임에서 문서 추가
database.add_documents(documents)

['0b0ea7a1-cc68-4045-8580-4e36ecd71ea5',
 '903fa773-d224-49cf-80ba-adea8be2879d',
 '7f61278f-ede3-48ea-b9b4-0bfcf3ef9783',
 'a0855d36-33a2-47bb-a964-9811c04e9da7',
 '0a7ddfe5-4a27-4a7f-aea9-c7685480a6be',
 '5a2d33e2-8dff-465e-959c-b5776f390e7f',
 '152eb3f6-d2a6-4b69-bec6-a39d21c826eb',
 'b86782e1-758b-422c-8305-3d5019d30735',
 '9882a115-8895-4506-a04a-4af74150d23a',
 '1969a45b-a3c3-4af9-8f60-9f82e789907a']

* 입력된 데이터 조회

In [9]:
database.get()

{'ids': ['0a7ddfe5-4a27-4a7f-aea9-c7685480a6be',
  '0b0ea7a1-cc68-4045-8580-4e36ecd71ea5',
  '152eb3f6-d2a6-4b69-bec6-a39d21c826eb',
  '1969a45b-a3c3-4af9-8f60-9f82e789907a',
  '5a2d33e2-8dff-465e-959c-b5776f390e7f',
  '7f61278f-ede3-48ea-b9b4-0bfcf3ef9783',
  '903fa773-d224-49cf-80ba-adea8be2879d',
  '9882a115-8895-4506-a04a-4af74150d23a',
  'a0855d36-33a2-47bb-a964-9811c04e9da7',
  'b86782e1-758b-422c-8305-3d5019d30735'],
 'embeddings': None,
 'metadatas': [None, None, None, None, None, None, None, None, None, None],
 'documents': ['아르바이트를 하고 있는데 지원할 수 있나요?\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.\n다만, 특수형태근로자의 경우 해촉증명서, 퇴사 사실확인증명원(사업주, 근로자 직인/서명포함)을 제출하는 경우 훈련장려금 지급이 가능하므로,\n고용형태 관련 세부사항은 거주지 관할 고용센터(HRD-Net 확인) 또는 훈련 권역별 고용센터로 문의하시기 바랍니다.\n※ 수도권: 성남고용센터\n충남/충북: 대전고용센터\n대구/경북: 대구고용센터\n전남/전북: 광주고용센터\n부산/경남: 부산고용센터\n- 고용노동부 대표번호 1350, 거주지 근처 지도검색, HRD-Net 메인 하단 지역별 

## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* DB, 테이블 생성

In [2]:
# DB 생성
path = './db_chatlog/db_chatlog.db'
conn = sqlite3.connect(path)

# 테이블 생성
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS history (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL NOT NULL,
    sim2 REAL NOT NULL,
    sim3 REAL NOT NULL,
    answer TEXT NOT NULL
)
''')

conn.commit()
conn.close()

* 모델 선언

In [37]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# ---------------------------------------------------------------
# 아래 코드가 다시 실행되어야 메모리가 초기화 됨

# 대화 메모리 초기화
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer", 
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, 
                                           return_source_documents=True,  output_key="answer")

In [48]:
# 질문
query = "지원하는데 나이 제한이 있나?"   # 질문할 문장

# 유사도 점수
result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
sim1 = round(result[0][1], 5)
sim2 = round(result[1][1], 5)
sim3 = round(result[2][1], 5)

# 현재 시간
dt = datetime.now()
dt = dt.strftime('%Y-%m-%d %H:%M:%S')

# 답변
result = qa(query)
answer = result["answer"]

# # history에 insert
conn = sqlite3.connect(path)

# # ② 작업 : to_sql
data = pd.DataFrame({'datetime': [dt], 'query': [query], 'sim1':[sim1], 'sim2':[sim2], 'sim3':[sim3], 'answer':[answer]})
data.to_sql('history', conn, if_exists='append', index=False)

# # ③ 연결 종료
conn.close()

In [45]:
# DB 입력 확인
conn = sqlite3.connect(path)
df = pd.read_sql('SELECT * FROM history', conn)
display(df)
conn.close()

,id,datetime,query,sim1,sim2,sim3,answer
0,1,2024-05-23 12:31:28,지원하는데 나이 제한이 있나요?,0.27265,0.31602,0.33948,"네, 해당 교육 과정은 34세 이하를 대상으로 하며, 모집 시점에 35세라도 해당연..."
1,2,2024-05-23 12:32:56,지원하는데 나이 제한이 있나요?,0.27269,0.31589,0.33929,"본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 다만, 모집 시점에 35세..."
2,3,2024-05-23 12:33:29,지원하는데 나이 제한이 있나?,0.27144,0.31428,0.33725,"네, 본 교육 과정은 34세 이하를 대상으로 하며, 모집시점에 35세라도 해당 연도..."
